In [2]:
!python -V

Python 3.9.12


In [1]:
import pandas as pd

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

In [19]:
def read_dataframe(filename,train=True):
 
    df = pd.read_parquet(filename)
    numberOfColumns = df.shape[1]
    df_unfiltered = df.shape[0]
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    std = df.duration.std()
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df_filtered = df.shape[0]
   
    if (train):
        print(f'Number of columns in the dataframe = {numberOfColumns}')
        print(f'duration std before filtering outliers: {std}')
        print(f'percentage of records left: {df_filtered/df_unfiltered}')
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet',False)

Number of columns in the dataframe = 19
duration std before filtering outliers: 42.594351241920904
percentage of records left: 0.9812202822125979


In [11]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical ].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical ].to_dict(orient='records')
X_val = dv.transform(val_dicts)

## one hot encoding

In [12]:
X_train.shape

(3009173, 515)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## training and evaluating linear regression model

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_predtrain = lr.predict(X_train)
print(f'RMSE on training = {root_mean_squared_error(y_train, y_predtrain)}')

RMSE on training = 7.649261027864015


In [16]:
y_pred = lr.predict(X_val)
print(f'RMSE on validation = {root_mean_squared_error(y_val, y_pred)}')

RMSE on validation = 7.811832425559196
